# Storm Data Science Plan

This notebook looks into our weather data in order to find storm periods and correlate them to extremes. 

In [ ]:
import numpy as np
import matplotlib.dates
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits import mplot3d
import pandas as pd
from datetime import datetime
import seaborn as sns
import hvplot.pandas
import holoviews as hv
from holoviews import dim, opts
hv.extension('bokeh')

## Recomputed Plume Bending Data 

**2010**

In [ ]:
path = '/home/jovyan/data/covis_data/recomputed/angles_partialpts_2010.csv' 
df_rc2010 = pd.read_csv(path, sep=",")
df_rc2010['year'] = '2010'
df_rc2010['datetime'] = pd.to_datetime(df_rc2010.year, format='%Y') + pd.to_timedelta(df_rc2010.date - 1, unit='d')
df_rc2010['datetime'] = df_rc2010['datetime'].dt.round('1s')
df_rc2010 = df_rc2010.set_index('datetime')
df_rc2010.drop(['date', 'year'], axis=1,inplace=True)
df_rc2010 = df_rc2010.rename_axis(None)
#fix for azimuth angle, 90 - azimuth then recomputation for any values that aren't in between 0 and 360 
df_rc2010['azimuth2'] = df_rc2010.azimuth.rsub(90)
mask = (df_rc2010['azimuth2'] < 0) 
df_rc2010.azimuth2[mask]=df_rc2010.azimuth2[mask]+360
df_rc2010['azirads'] = df_rc2010['azimuth']*(np.pi/180)
df_rc2010['incrads'] = df_rc2010['inclination']*(np.pi/180)
df_rc2010 = df_rc2010.resample('h').mean()
df_rc2010.head(20)

In [ ]:
r = df_rc2010['r_value'] = 1
theta = df_rc2010['incrads']
phi = df_rc2010['azirads']
df_rc2010['rc_north'] = r * np.sin(theta) * np.cos(phi)
df_rc2010['rc_east'] = r * np.sin(theta) * np.sin(phi)
df_rc2010.drop(['r_value'], axis=1,inplace=True)
df_rc2010.head(50)

**2011**

In [ ]:
path = '/home/jovyan/data/covis_data/recomputed/angles_partialpts_2011.csv' 
df_rc2011 = pd.read_csv(path, sep=",")
df_rc2011['year'] = '2011'
df_rc2011['datetime'] = pd.to_datetime(df_rc2011.year, format='%Y') + pd.to_timedelta(df_rc2011.date - 1, unit='d')
df_rc2011['datetime'] = df_rc2011['datetime'].dt.round('1s')
df_rc2011 = df_rc2011.set_index('datetime')
df_rc2011.drop(['date', 'year'], axis=1,inplace=True)
df_rc2011 = df_rc2011.rename_axis(None)
#fix for azimuth angle, 90 - azimuth then recomputation for any values that aren't in between 0 and 360 
df_rc2011['azimuth2'] = df_rc2011.azimuth.rsub(90)
mask = (df_rc2011['azimuth2'] < 0) 
df_rc2011.azimuth2[mask]=df_rc2011.azimuth2[mask]+360
df_rc2011['azirads'] = df_rc2011['azimuth']*(np.pi/180)
df_rc2011['incrads'] = df_rc2011['inclination']*(np.pi/180)
df_rc2011 = df_rc2011.resample('h').mean()
df_rc2011.head(20)

In [ ]:
r = df_rc2011['r_value'] = 1
theta = df_rc2011['azimuth']
phi = df_rc2011['inclination']
df_rc2011['rc_north'] = r * np.sin(theta) * np.cos(phi)
df_rc2011['rc_east'] = r * np.sin(theta) * np.sin(phi)
df_rc2011.drop(['r_value'], axis=1, inplace=True)
df_rc2011

## Weather Data 

**2010 C46036**

In [ ]:
path = '/home/jovyan/data/covis_data/weather_data_for_plotting_2010_C46036.csv' 
df_wd2010c46036 = pd.read_csv(path, sep=",")
df_wd2010c46036['year']= df_wd2010c46036['year'].astype(int).astype(str)
df_wd2010c46036['month']= df_wd2010c46036['month'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_wd2010c46036['day']= df_wd2010c46036['day'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_wd2010c46036['hour']= df_wd2010c46036['hour'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_wd2010c46036['minute']=df_wd2010c46036['minute'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_wd2010c46036['datetime'] = df_wd2010c46036['year'] + df_wd2010c46036['month'] + df_wd2010c46036['day'] +\
'T' + df_wd2010c46036['hour']+ ':' + df_wd2010c46036['minute']
df_wd2010c46036['datetime'] = pd.to_datetime(df_wd2010c46036['datetime'])
df_wd2010c46036 = df_wd2010c46036.set_index('datetime')
df_wd2010c46036 = df_wd2010c46036[['jday', 'wv_height', 'wnd_dir', 'wnd_spd', 'wnd_gspd', 'atm_prs', 'air_temp']]
df_wd2010c46036 = df_wd2010c46036.rename_axis(None)
df_wd2010c46036['wndrads'] = df_wd2010c46036['wnd_dir']*(np.pi/180)
df_wd2010c46036 = df_wd2010c46036.resample('h').mean()
df_wd2010c46036.head()

In [ ]:
r = df_wd2010c46036['r_value'] = 1
theta = df_wd2010c46036['wndrads']
phi = df_wd2010c46036['wndrads']
df_wd2010c46036['wd_north'] = r * np.sin(theta) * np.cos(phi)
df_wd2010c46036['wd_east'] = r * np.sin(theta) * np.sin(phi)
df_wd2010c46036.drop(['r_value'], axis=1, inplace=True)
df_wd2010c46036

**2011 C46036**

In [ ]:
path = '/home/jovyan/data/covis_data/weather_data_for_plotting_2011_C46036.csv'
df_wd2011c46036 = pd.read_csv(path, sep=",")
df_wd2011c46036['year']= df_wd2011c46036['year'].astype(int).astype(str)
df_wd2011c46036['month']= df_wd2011c46036['month'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_wd2011c46036['day']= df_wd2011c46036['day'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_wd2011c46036['hour']= df_wd2011c46036['hour'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_wd2011c46036['minute']=df_wd2011c46036['minute'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_wd2011c46036['datetime'] = df_wd2011c46036['year'] + df_wd2011c46036['month'] + df_wd2011c46036['day'] +\
'T' + df_wd2011c46036['hour']+ ':' + df_wd2011c46036['minute']
df_wd2011c46036['datetime'] = pd.to_datetime(df_wd2011c46036['datetime'])
df_wd2011c46036 = df_wd2011c46036.set_index('datetime')
df_wd2011c46036 = df_wd2011c46036[['jday', 'wv_height', 'wnd_dir', 'wnd_spd', 'wnd_gspd', 'atm_prs', 'air_temp']]
df_wd2011c46036 = df_wd2011c46036.rename_axis(None)
df_wd2011c46036['wndrads'] = df_wd2011c46036['wnd_dir']*(np.pi/180)
df_wd2011c46036 = df_wd2011c46036.resample('h').mean()
df_wd2011c46036.head()

In [ ]:
r = df_wd2011c46036['r_value'] = 1
theta = df_wd2011c46036['wndrads']
phi = df_wd2011c46036['wndrads']
df_wd2011c46036['wd_north'] = r * np.sin(theta) * np.cos(phi)
df_wd2011c46036['wd_east'] = r * np.sin(theta) * np.sin(phi)
df_wd2011c46036.drop(['r_value'], axis=1, inplace=True)
df_wd2011c46036

**2010 Tillamook**

In [ ]:
path = '/home/jovyan/data/covis_data/weather_data_for_plotting_2010_Tillamook.csv' 
df_wd2010Tillamook = pd.read_csv(path, sep=",")
df_wd2010Tillamook['year']= df_wd2010Tillamook['year'].astype(int).astype(str)
df_wd2010Tillamook['month']= df_wd2010Tillamook['month'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_wd2010Tillamook['day']= df_wd2010Tillamook['day'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_wd2010Tillamook['hour']= df_wd2010Tillamook['hour'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_wd2010Tillamook['minute']=df_wd2010Tillamook['minute'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_wd2010Tillamook['wv_height'] = df_wd2010Tillamook['wv_height'].astype('float64')
df_wd2010Tillamook['wnd_dir'] = df_wd2010Tillamook['wnd_dir'].astype('float64')
df_wd2010Tillamook['wnd_spd'] = df_wd2010Tillamook['wnd_spd'].astype('float64')
df_wd2010Tillamook['wnd_gspd'] = df_wd2010Tillamook['wnd_gspd'].astype('float64')
df_wd2010Tillamook['atm_prs'] = df_wd2010Tillamook['atm_prs'].astype('float64')
df_wd2010Tillamook['air_temp'] = df_wd2010Tillamook['air_temp'].astype('float64')
df_wd2010Tillamook['datetime'] = df_wd2010Tillamook['year'] + df_wd2010Tillamook['month'] + df_wd2010Tillamook['day'] +\
'T' + df_wd2010Tillamook['hour']+ ':' + df_wd2010Tillamook['minute']
df_wd2010Tillamook['datetime'] = pd.to_datetime(df_wd2010Tillamook['datetime'])
df_wd2010Tillamook = df_wd2010Tillamook.set_index('datetime')
df_wd2010Tillamook = df_wd2010Tillamook[['jday', 'wv_height', 'wnd_dir', 'wnd_spd', 'wnd_gspd', 'atm_prs', 'air_temp']]
df_wd2010Tillamook = df_wd2010Tillamook.rename_axis(None)
df_wd2010Tillamook['wndrads'] = df_wd2010Tillamook['wnd_dir']*(np.pi/180)
df_wd2010Tillamook = df_wd2010Tillamook.resample('h').mean()
df_wd2010Tillamook.head()

In [ ]:
r = df_wd2010Tillamook['r_value'] = 1
theta = df_wd2010Tillamook['wnd_dir']
phi = df_wd2010Tillamook['wnd_dir']
df_wd2010Tillamook['wd_north'] = r * np.sin(theta) * np.cos(phi)
df_wd2010Tillamook['wd_east'] = r * np.sin(theta) * np.sin(phi)
df_wd2010Tillamook.drop(['r_value'], axis=1, inplace=True)
df_wd2010Tillamook

**2011 Tillamook**

In [ ]:
path = '/home/jovyan/data/covis_data/weather_data_for_plotting_2011_Tillamook.csv' 
df_wd2011Tillamook = pd.read_csv(path, sep=",", engine='python')
df_wd2011Tillamook= df_wd2011Tillamook.dropna()
df_wd2011Tillamook['year']= df_wd2011Tillamook['year'].astype(int).astype(str)
df_wd2011Tillamook['month']= df_wd2011Tillamook['month'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_wd2011Tillamook['day']= df_wd2011Tillamook['day'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_wd2011Tillamook['hour']= df_wd2011Tillamook['hour'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_wd2011Tillamook['minute']=df_wd2011Tillamook['minute'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_wd2011Tillamook['wv_height'] = df_wd2011Tillamook['wv_height'].astype('float64')
df_wd2011Tillamook['wnd_dir'] = df_wd2011Tillamook['wnd_dir'].astype('float64')
df_wd2011Tillamook['wnd_spd'] = df_wd2011Tillamook['wnd_spd'].astype('float64')
df_wd2011Tillamook['wnd_gspd'] = df_wd2011Tillamook['wnd_gspd'].astype('float64')
df_wd2011Tillamook['atm_prs'] = df_wd2011Tillamook['atm_prs'].astype('float64')
df_wd2011Tillamook['air_temp'] = df_wd2011Tillamook['air_temp'].astype('float64')
df_wd2011Tillamook['datetime'] = df_wd2011Tillamook['year'] + df_wd2011Tillamook['month'] + df_wd2011Tillamook['day'] +\
'T' + df_wd2011Tillamook['hour']+ ':' + df_wd2011Tillamook['minute']
df_wd2011Tillamook['datetime'] = pd.to_datetime(df_wd2011Tillamook['datetime'])
df_wd2011Tillamook = df_wd2011Tillamook.set_index('datetime')
df_wd2011Tillamook = df_wd2011Tillamook[['jday', 'wv_height', 'wnd_dir', 'wnd_spd', 'wnd_gspd', 'atm_prs', 'air_temp']]
df_wd2011Tillamook = df_wd2011Tillamook.rename_axis(None)
df_wd2011Tillamook['wndrads'] = df_wd2011Tillamook['wnd_dir']*(np.pi/180)
df_wd2011Tillamook = df_wd2011Tillamook.resample('h').mean()
df_wd2011Tillamook.head()

In [ ]:
r = df_wd2011Tillamook['r_value'] = 1
theta = df_wd2011Tillamook['wnd_dir']
phi = df_wd2011Tillamook['wnd_dir']
df_wd2011Tillamook['wd_north'] = r * np.sin(theta) * np.cos(phi)
df_wd2011Tillamook['wd_east'] = r * np.sin(theta) * np.sin(phi)
df_wd2011Tillamook.drop(['r_value'], axis=1, inplace=True)
df_wd2011Tillamook

## Mooring Data

**Mooring NE 005**

In [ ]:
path = '/home/jovyan/data/covis_data/mooring_data/MooringNE_005mab_CurrentMeter_2010_python.csv'
df_NE_005mab2010 = pd.read_csv(path, sep=",")
df_NE_005mab2010['datetime'] = matplotlib.dates.num2date(df_NE_005mab2010['time'], tz=None)
df_NE_005mab2010 = df_NE_005mab2010.set_index('datetime')
df_NE_005mab2010 = df_NE_005mab2010.resample('T').mean()
df_NE_005mab2010.drop(['time'], axis=1,inplace=True)
df_NE_005mab2010['angle'] = np.arctan2(df_NE_005mab2010['current_nc'],df_NE_005mab2010['current_ec'])
df_NE_005mab2010['angle'] = np.degrees(df_NE_005mab2010['angle'])
#fix for angle, 90 - angle then recomputation for any values that aren't in between 0 and 360 
df_NE_005mab2010['angle2'] = 90 - df_NE_005mab2010['angle'] 
mask = (df_NE_005mab2010['angle2'] < 0) 
df_NE_005mab2010.angle2[mask]=df_NE_005mab2010.angle2[mask]+360
df_NE_005mab2010

In [ ]:
path = '/home/jovyan/data/covis_data/mooring_data/MooringNE_005mab_CurrentMeter_2011_python_test2.csv'
df_NE_005mab2011 = pd.read_csv(path, sep=",")
df_NE_005mab2011['datetime'] = matplotlib.dates.num2date(df_NE_005mab2011['time'], tz=None)
df_NE_005mab2011 = df_NE_005mab2011.set_index('datetime')
df_NE_005mab2011 = df_NE_005mab2011.resample('T').mean()
df_NE_005mab2011.drop(['time'], axis=1,inplace=True)
df_NE_005mab2011['angle'] = np.arctan2(df_NE_005mab2011['current_nc'],df_NE_005mab2011['current_ec'])
df_NE_005mab2011['angle'] = np.degrees(df_NE_005mab2011['angle'])
#fix for angle, 90 - angle then recomputation for any values that aren't in between 0 and 360 
df_NE_005mab2011['angle2'] = 90 - df_NE_005mab2011['angle'] 
mask = (df_NE_005mab2011['angle2'] < 0) 
df_NE_005mab2011.angle2[mask]=df_NE_005mab2011.angle2[mask]+360
df_NE_005mab2011 = df_NE_005mab2011['2011-05-20 21:28:00+00:00' : "2011-12-31 23:59:00+00:00"]
df_NE_005mab2011

**Mooring NE 050**

In [ ]:
path = '/home/jovyan/data/covis_data/mooring_data/MooringNE_050mab_CurrentMeter_2010_python.csv'
df_NE_050mab2010 = pd.read_csv(path, sep=",")
df_NE_050mab2010['datetime'] = matplotlib.dates.num2date(df_NE_050mab2010['time'], tz=None)
df_NE_050mab2010 = df_NE_050mab2010.set_index('datetime')
df_NE_050mab2010 = df_NE_050mab2010.resample('T').mean()
df_NE_050mab2010.drop(['time'], axis=1,inplace=True)
df_NE_050mab2010['angle'] = np.arctan2(df_NE_050mab2010['current_nc'],df_NE_050mab2010['current_ec'])
df_NE_050mab2010['angle'] = np.degrees(df_NE_050mab2010['angle'])
df_NE_050mab2010['angle2'] = 90 - df_NE_050mab2010['angle'] 
mask = (df_NE_050mab2010['angle2'] < 0) 
df_NE_050mab2010.angle2[mask]=df_NE_050mab2010.angle2[mask]+360
df_NE_050mab2010.head(5)

In [ ]:
path = '/home/jovyan/data/covis_data/mooring_data/MooringNE_050mab_CurrentMeter_2011_python_test2.csv'
df_NE_050mab2011 = pd.read_csv(path, sep=",")
df_NE_050mab2011['datetime'] = matplotlib.dates.num2date(df_NE_050mab2011['time'], tz=None)
df_NE_050mab2011 = df_NE_050mab2011.set_index('datetime')
df_NE_050mab2011 = df_NE_050mab2011.resample('T').mean()
df_NE_050mab2011.drop(['time'], axis=1,inplace=True)
df_NE_050mab2011['angle'] = np.arctan2(df_NE_050mab2011['current_nc'],df_NE_050mab2011['current_ec'])
df_NE_050mab2011['angle'] = np.degrees(df_NE_050mab2011['angle'])
#fix for angle, 90 - angle then recomputation for any values that aren't in between 0 and 360 
df_NE_050mab2011['angle2'] = 90 - df_NE_050mab2011['angle'] 
mask = (df_NE_050mab2011['angle2'] < 0) 
df_NE_050mab2011.angle2[mask]=df_NE_050mab2011.angle2[mask]+360
df_NE_050mab2011 = df_NE_050mab2011['2011-01-06 19:25:00+00:00' : '2011-12-31 23:59:00+00:00']
df_NE_050mab2011.head()

## Combining Data

**2010 C46036**

In [ ]:
rc2010c = pd.merge(df_wd2010c46036, df_rc2010,how='inner', indicator=True, left_index=True, right_index=True, suffixes=('_B', '_G'))

In [ ]:
df_covis_rc2010c = pd.DataFrame()
df_covis_rc2010c = rc2010c[rc2010c['_merge'] == 'both']
del df_covis_rc2010c['_merge']
df_covis_rc2010c = df_covis_rc2010c.dropna()
del df_covis_rc2010c['jday']
df_covis_rc2010c.head()

**2010 Tillamook**

In [ ]:
rc2010T = pd.merge(df_wd2010Tillamook, df_rc2010,how='inner', indicator=True, left_index=True, right_index=True, suffixes=('_B', '_G'))

In [ ]:
df_covis_rc2010T = pd.DataFrame()
df_covis_rc2010T = rc2010T[rc2010T['_merge'] == 'both']
del df_covis_rc2010T['_merge']
df_covis_rc2010T.drop(['wv_height'], axis=1,inplace=True)
df_covis_rc2010T.drop(['atm_prs'], axis=1,inplace=True)
df_covis_rc2010T.drop(['wnd_gspd'], axis=1,inplace=True)
df_covis_rc2010T = df_covis_rc2010T.dropna()
del df_covis_rc2010T['jday']
df_covis_rc2010T.head()

**2011 C46036**

In [ ]:
rc2011c = pd.merge(df_wd2011c46036, df_rc2011,how='inner', indicator=True, left_index=True, right_index=True, suffixes=('_B', '_G'))

In [ ]:
df_covis_rc2011c = pd.DataFrame()
df_covis_rc2011c = rc2011c[rc2011c['_merge'] == 'both']
del df_covis_rc2011c['_merge']
df_covis_rc2011c = df_covis_rc2011c.dropna()
del df_covis_rc2011c['jday']
df_covis_rc2011c.head()

**2011 Tillamook**

In [ ]:
rc2011T = pd.merge(df_wd2011Tillamook, df_rc2011,how='inner', indicator=True, left_index=True, right_index=True, suffixes=('_B', '_G'))

In [ ]:
df_covis_rc2011T = pd.DataFrame()
df_covis_rc2011T = rc2011T[rc2011T['_merge'] == 'both']
del df_covis_rc2011T['_merge']
df_covis_rc2011T.drop(['wv_height'], axis=1,inplace=True)
df_covis_rc2011T.drop(['atm_prs'], axis=1,inplace=True)
df_covis_rc2011T.drop(['wnd_gspd'], axis=1,inplace=True)
df_covis_rc2011T = df_covis_rc2011T.dropna()
del df_covis_rc2011T['jday']
df_covis_rc2011T.head()

## Find Extremes

From our previous results, we've determined an extreme inclination to be >40 degrees. This was only present so far in our 2011 data, so for now this data set will be focused on. 

**C46036**

In [ ]:
#isolate the dates that have extreme inclinations
df_covisrc2011cextreme=df_covis_rc2011c.loc[df_covis_rc2011c['inclination']>40]
df_covisrc2011cextreme

**Tillamook**

In [ ]:
df_covisrc2011Textreme=df_covis_rc2011T.loc[df_covis_rc2011T['inclination']>40]
df_covisrc2011Textreme

**Combined**

In [ ]:
extremes = pd.merge(df_covisrc2011cextreme, df_covisrc2011Textreme,how='inner', indicator=True, left_index=True, right_index=True, suffixes=('_C', '_T'))

In [ ]:
df_covis_extremes = pd.DataFrame()
df_covis_extremes = extremes[extremes['_merge'] == 'both']
del df_covis_extremes['_merge']
df_covis_extremes.drop(['wndrads_C'], axis=1,inplace=True)
df_covis_extremes.drop(['wndrads_T'], axis=1,inplace=True)
df_covis_extremes.drop(['azirads_C'], axis=1,inplace=True)
df_covis_extremes.drop(['incrads_C'], axis=1,inplace=True)
df_covis_extremes.drop(['azirads_T'], axis=1,inplace=True)
df_covis_extremes.drop(['incrads_T'], axis=1,inplace=True)
df_covis_extremes.drop(['wv_height'], axis=1,inplace=True)
df_covis_extremes.drop(['wnd_gspd'], axis=1,inplace=True)
df_covis_extremes.drop(['atm_prs'], axis=1,inplace=True)
df_covis_extremes = df_covis_extremes.dropna()
df_covis_extremes.head()

In [ ]:
cols = df_covis_extremes.columns.tolist()
cols

In [ ]:
df_covis_extremes = df_covis_extremes[['wnd_dir_C','wnd_dir_T','wnd_spd_C','wnd_spd_T','air_temp_C','air_temp_T','wd_north_C','wd_north_T','wd_east_C','wd_east_T','inclination_C','inclination_T','azimuth_C','azimuth_T','azimuth2_C','azimuth2_T','rc_north_C','rc_north_T','rc_east_C','rc_east_T']]
df_covis_extremes